# Cross-Session Memory System Demo

This notebook demonstrates the **cross-session memory system** that learns from past queries.

## Overview

The memory system has two tiers:

### **Tier 1: Manual Lessons** (config/lessons_learned.yaml)
- High-confidence patterns that are manually configured
- Examples: table prefix rules, known column mappings
- **Confidence: 0.90-1.00**

### **Tier 2: Auto-Learned Lessons** (memory/learned_lessons.json)
- Patterns automatically extracted from successful/failed queries
- Learns from:
  - Error recovery (table not found → added prefix → success)
  - User corrections ("Use PROD_ prefix")
  - Successful query patterns
- **Confidence: 0.60-0.90** (increases with successful use)

## What You'll Learn

1. How to configure manual lessons
2. How the system learns from errors
3. How the system learns from corrections
4. How lessons are applied to new queries
5. How to inspect and manage learned patterns

## Setup and Imports

In [ ]:
import sys
from pathlib import Path
import json
import yaml

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from src import Text2SQLAgent
from src.memory import (
    lesson_repository,
    TableMapper,
    LessonLearner,
    TableMappingLesson,
)
from src.utils import setup_logger

logger = setup_logger(__name__)

print("✅ Imports successful!")

---

# Part 1: Understanding Manual Lessons

Manual lessons are high-confidence patterns stored in `config/lessons_learned.yaml`.

## View Current Manual Lessons

In [ ]:
# Read the manual lessons configuration
config_path = Path.cwd().parent / "config" / "lessons_learned.yaml"

if config_path.exists():
    with open(config_path, 'r') as f:
        manual_config = yaml.safe_load(f)
    
    print("📚 Manual Lessons Configuration:")
    print("=" * 70)
    
    # Table mappings
    if manual_config.get("table_mappings"):
        print(f"\n✓ Table Mappings: {len(manual_config['table_mappings'])}")
        for mapping in manual_config["table_mappings"]:
            print(f"  - {mapping.get('content')}")
            print(f"    Confidence: {mapping.get('confidence', 0):.0%}")
    
    # Column mappings
    if manual_config.get("column_mappings"):
        print(f"\n✓ Column Mappings: {len(manual_config['column_mappings'])}")
    
    # Error patterns
    if manual_config.get("error_patterns"):
        print(f"\n✓ Error Patterns: {len(manual_config['error_patterns'])}")
        for pattern in manual_config["error_patterns"]:
            print(f"  - {pattern.get('content')}")
            print(f"    Fix: {pattern.get('suggested_fix')}")
else:
    print("⚠️ No manual lessons configuration found")
    print(f"Expected at: {config_path}")

## Load Lessons from Repository

In [ ]:
# Get all lessons (manual + learned)
all_lessons = lesson_repository.get_all_lessons()

print(f"📖 Total Lessons in Repository: {len(all_lessons)}")
print("=" * 70)

# Count by type
from collections import Counter
lesson_types = Counter(lesson.type.value for lesson in all_lessons)

for lesson_type, count in lesson_types.items():
    print(f"  {lesson_type}: {count}")

# Show top lessons
print("\n🏆 Top 5 Highest Confidence Lessons:")
print("-" * 70)
sorted_lessons = sorted(all_lessons, key=lambda x: x.confidence, reverse=True)[:5]

for i, lesson in enumerate(sorted_lessons, 1):
    print(f"{i}. [{lesson.type.value}] {lesson.content}")
    print(f"   Confidence: {lesson.confidence:.0%}, Source: {lesson.source}")
    if lesson.times_applied > 0:
        print(f"   Usage: {lesson.times_applied} times, Success Rate: {lesson.success_rate:.0%}")

---

# Part 2: Table Mapper in Action

The `TableMapper` applies learned transformations to table names.

In [ ]:
# Initialize table mapper
mapper = TableMapper()

# Example table names from schema
schema_tables = [
    "Customers",
    "Orders",
    "Products",
    "SalesData",
    "Transactions",
]

print("🔄 Table Name Transformations")
print("=" * 70)
print(f"{'Original Table':<20} → {'Transformed Table':<20} {'Confidence':<12} {'Rule'}")
print("-" * 70)

for table in schema_tables:
    transformed = mapper.transform(table)
    info = mapper.get_transformation_info(table)
    
    if info:
        print(f"{table:<20} → {transformed:<20} {info['confidence']:.0%}          {info['rule'][:30]}")
    else:
        print(f"{table:<20} → {transformed:<20} {'N/A':<12} No transformation")

## Preview Transformations

Before applying transformations, you can preview what will happen:

In [ ]:
# Preview transformations for a set of tables
preview = mapper.preview_transformations(schema_tables)

print("\n🔍 Transformation Preview:")
print("=" * 70)

for item in preview:
    if item['original'] != item['transformed']:
        print(f"\n✓ {item['original']} → {item['transformed']}")
        print(f"  Confidence: {item['confidence']:.0%}")
        print(f"  Source: {item['source']}")
        print(f"  Rule: {item['rule']}")
        if item['success_rate'] > 0:
            print(f"  Past Success Rate: {item['success_rate']:.0%}")

---

# Part 3: Learning from Errors

The system automatically learns when an error is recovered.

## Scenario: Table Not Found Error

**What happens:**
1. User asks: "Show me all customers"
2. System generates: `SELECT * FROM Customers`
3. BigQuery error: "Table `project.dataset.Customers` not found"
4. System retries with: `SELECT * FROM PROD_Customers`
5. Success! ✓

**Lesson learned:**
- `Customers` → `PROD_Customers` (add PROD_ prefix)
- Confidence: 0.80 (auto-learned)
- Saved to `memory/learned_lessons.json`

In [ ]:
# Check if any lessons were auto-learned
learned_path = Path.cwd().parent / "memory" / "learned_lessons.json"

if learned_path.exists():
    with open(learned_path, 'r') as f:
        learned_lessons = json.load(f)
    
    print(f"🧠 Auto-Learned Lessons: {len(learned_lessons)}")
    print("=" * 70)
    
    if learned_lessons:
        for lesson in learned_lessons[:5]:  # Show first 5
            print(f"\n📝 {lesson.get('content')}")
            print(f"   Type: {lesson.get('type')}")
            print(f"   Confidence: {lesson.get('confidence', 0):.0%}")
            print(f"   Source: {lesson.get('source')}")
            
            if lesson.get('times_applied', 0) > 0:
                success_rate = lesson['times_successful'] / lesson['times_applied']
                print(f"   Applied: {lesson['times_applied']} times")
                print(f"   Success Rate: {success_rate:.0%}")
            
            if lesson.get('learned_from_sessions'):
                print(f"   Learned from: {len(lesson['learned_from_sessions'])} session(s)")
    else:
        print("\nNo auto-learned lessons yet. Run some queries to start learning!")
else:
    print("📁 No learned lessons file found yet.")
    print("Run queries with the agent to start building institutional knowledge!")

---

# Part 4: Learning from User Corrections

When users provide corrections, the system extracts patterns.

## Example Corrections

**Correction 1:** "Use PROD_ prefix for all tables"
- → Learns: Add PROD_ prefix pattern

**Correction 2:** "customer_id should be cust_id"
- → Learns: Column mapping (customer_id → cust_id)

**Correction 3:** "Join on Orders.customer_id = Customers.id"
- → Learns: Specific join pattern

## Simulate Learning from a Correction

Let's manually create a lesson to see how it works:

In [ ]:
import uuid

# Create a new table mapping lesson
new_lesson = TableMappingLesson(
    id=str(uuid.uuid4()),
    content="Sales table requires DWH_ prefix in data warehouse",
    schema_name="Sales",
    actual_name="DWH_Sales",
    prefix="DWH_",
    transformation_rule="Add 'DWH_' prefix to Sales table",
    confidence=0.85,
    source="manual",  # Could be "correction" or "auto_learned"
    applicable_to=["Sales"],
)

# Add to repository
lesson_repository.add_lesson(new_lesson, save=True)

print("✅ New lesson added to repository!")
print(f"\nLesson Details:")
print(f"  Content: {new_lesson.content}")
print(f"  Transformation: {new_lesson.schema_name} → {new_lesson.actual_name}")
print(f"  Confidence: {new_lesson.confidence:.0%}")

# Test the transformation
result = mapper.transform("Sales")
print(f"\n🔄 Test transformation:")
print(f"  Input: Sales")
print(f"  Output: {result}")

---

# Part 5: End-to-End Demo with Agent

Let's see how the memory system integrates with the full agent workflow.

## Initialize Agent

In [ ]:
try:
    # Initialize agent (this loads schema and memory)
    agent = Text2SQLAgent()
    
    print("✅ Agent initialized successfully!")
    print(f"   Tables loaded: {len(agent.schema.tables)}")
    print(f"   Lessons loaded: {len(lesson_repository.get_all_lessons())}")
    
except Exception as e:
    print(f"⚠️  Could not initialize agent: {e}")
    print("\nThis demo requires proper configuration.")
    print("See AGENT_GUIDE.md for setup instructions.")

## Query with Memory

When you run a query, the agent automatically:
1. Applies learned table transformations
2. Includes relevant lessons in the LLM prompt
3. After success, learns new patterns

In [ ]:
# Example query
query = "Show me top 10 customers by order count"

print(f"💬 Query: \"{query}\"")
print("\n🔄 Processing with memory system...")
print("-" * 70)

try:
    # Execute query (memory is automatically applied)
    result = agent.query(query, execute=False)  # Set execute=True to run on BigQuery
    
    if result["success"]:
        print("\n✅ Query generated successfully!")
        print("\n📝 Generated SQL:")
        print("-" * 70)
        print(result["sql"])
        print("-" * 70)
        
        # Check if transformations were applied
        if "PROD_" in result["sql"]:
            print("\n🎯 Memory system applied: PROD_ prefix transformation")
        
        print("\n💡 After this query, any new patterns will be automatically learned!")
    else:
        print(f"\n❌ Query failed: {result.get('error')}")
        print(f"Message: {result.get('message')}")

except Exception as e:
    print(f"\n❌ Error: {e}")

---

# Part 6: Inspecting Learned Patterns

Over time, the system builds up institutional knowledge.

## View Lessons by Confidence

In [ ]:
# Get all lessons sorted by confidence
all_lessons = lesson_repository.get_all_lessons()
sorted_by_confidence = sorted(all_lessons, key=lambda x: x.confidence, reverse=True)

print("📊 Lessons by Confidence Level")
print("=" * 70)

# High confidence (>90%)
high_conf = [l for l in sorted_by_confidence if l.confidence > 0.9]
print(f"\n🟢 High Confidence (>90%): {len(high_conf)} lessons")
for lesson in high_conf[:3]:
    print(f"  - {lesson.content} ({lesson.confidence:.0%})")

# Medium confidence (70-90%)
med_conf = [l for l in sorted_by_confidence if 0.7 <= l.confidence <= 0.9]
print(f"\n🟡 Medium Confidence (70-90%): {len(med_conf)} lessons")
for lesson in med_conf[:3]:
    print(f"  - {lesson.content} ({lesson.confidence:.0%})")

# Low confidence (<70%)
low_conf = [l for l in sorted_by_confidence if l.confidence < 0.7]
print(f"\n🔴 Low Confidence (<70%): {len(low_conf)} lessons")
for lesson in low_conf[:3]:
    print(f"  - {lesson.content} ({lesson.confidence:.0%})")

print("\n💡 Lessons with low confidence may need review or validation.")

## View Lessons by Usage

See which lessons are most frequently used:

In [ ]:
# Sort by usage frequency
sorted_by_usage = sorted(
    [l for l in all_lessons if l.times_applied > 0],
    key=lambda x: x.times_applied,
    reverse=True
)

print("📈 Most Frequently Used Lessons")
print("=" * 70)

if sorted_by_usage:
    for i, lesson in enumerate(sorted_by_usage[:10], 1):
        print(f"\n{i}. {lesson.content}")
        print(f"   Applied: {lesson.times_applied} times")
        print(f"   Success Rate: {lesson.success_rate:.0%}")
        print(f"   Current Confidence: {lesson.confidence:.0%}")
else:
    print("\nNo lessons have been applied yet.")
    print("Run more queries to build up usage statistics!")

---

# Part 7: Managing Lessons

You can view, update, and manage lessons.

## Export Lessons for Review

In [ ]:
# Export all lessons to a readable format
export_data = []

for lesson in all_lessons:
    export_data.append({
        "content": lesson.content,
        "type": lesson.type.value,
        "confidence": f"{lesson.confidence:.0%}",
        "source": lesson.source,
        "times_applied": lesson.times_applied,
        "success_rate": f"{lesson.success_rate:.0%}" if lesson.times_applied > 0 else "N/A",
    })

# Display as a formatted table
import pandas as pd

if export_data:
    df = pd.DataFrame(export_data)
    print("\n📋 All Lessons Summary")
    print("=" * 70)
    display(df)
    
    # Optionally save to CSV
    # df.to_csv('lessons_export.csv', index=False)
    # print("\n💾 Exported to lessons_export.csv")
else:
    print("No lessons to export.")

---

# Summary

## Key Takeaways

1. **Two-Tier System**: Manual (high confidence) + Auto-learned (adaptive)
2. **Automatic Learning**: System learns from errors and corrections without manual intervention
3. **Confidence Tracking**: Each lesson has a confidence score that adjusts with usage
4. **Seamless Integration**: Memory is automatically applied during query processing
5. **Transparent**: All lessons can be inspected, reviewed, and managed

## How It Helps

✅ **Solves the table prefix problem**: Automatically maps schema tables to actual database tables

✅ **Builds institutional knowledge**: Patterns are saved and reused across sessions

✅ **Improves over time**: Confidence increases with successful applications

✅ **Reduces errors**: Common mistakes are remembered and avoided

✅ **Transparent learning**: All learned patterns are visible and manageable

## Configuration Files

- **Manual lessons**: `config/lessons_learned.yaml`
- **Auto-learned lessons**: `memory/learned_lessons.json`

## Next Steps

1. Configure your manual lessons in `config/lessons_learned.yaml`
2. Run queries with the agent to start learning
3. Monitor learned patterns in `memory/learned_lessons.json`
4. Review and validate low-confidence lessons periodically
5. Promote successful auto-learned lessons to manual configuration